# Using Random Forest

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

##### Define Functions

In [37]:
def RandomForest(X_train, X_test, Y1_train, Y2_train,N_estimator):
    
    regressor = RandomForestRegressor(n_estimators = N_estimator, random_state = 0)
    regressor.fit(X_train, Y1_train)
    y1_pred = regressor.predict(X_test)
    regressor.fit(X_train, Y2_train)
    y2_pred = regressor.predict(X_test)
    
    return y1_pred, y2_pred

def separateData(dataset, testset):
    dataset = dataset[["RSSI A", "RSSI B", "RSSI C","x","y"]]
    testset = testset[["RSSI A", "RSSI B", "RSSI C","x","y"]]

    X_train = dataset.iloc[:, :3]
    X_train = X_train.values
    
    X_test = testset.iloc[:, :3]
    X_test = X_test.values
    
    Y1_train = dataset['x']
    Y1_train = Y1_train.values.reshape(-1,1)
    
    Y2_train = dataset['y']
    Y2_train = Y2_train.values.reshape(-1,1)   
    
    Y1_test = testset['x']
    Y1_test = Y1_test.values.reshape(-1,1)
    
    Y2_test = testset['y']
    Y2_test = Y2_test.values.reshape(-1,1)
    
    return X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test

def error(x_calc, x_real, y_calc, y_real):
    x = ( (x_calc[i] - x_real[i]) ** 2 + (y_calc[i] - y_real[i]) ** 2 ) ** 0.5        
    return x

def Average(lst):
    return sum(lst) / len(lst)

# For hyperparameter Tuning
def error1(x_calc, x_real, y_calc, y_real):
    x = ( (x_calc - x_real) ** 2 + (y_calc - y_real) ** 2 ) ** 0.5   
    return x
def geterror(y1_pred,Y1_test,y2_pred,Y2_test):
    lenght = list(range(0,len(Y1_test)))
    x_vec_0 = []
    for i in lenght:
        x_vec_0.append(error1(y1_pred[i],Y1_test[i],y2_pred[i],Y2_test[i])[0])  
    arr = np.asarray(x_vec_0)
    average_0 = sum(arr)/len(arr)
    return average_0

## Scenario 1

In [38]:
#read dataset for scenario 1
df_BLE = pd.read_csv('BLE_scenario1.csv')
df_Zigbee = pd.read_csv('Zigbee_scenario1.csv')
df_WiFi = pd.read_csv('WiFi_scenario1.csv')

In [39]:
# TO SPLIT THE WHOLE DATASET INTO TRAINING DATASET AND TESTING DATASET
from sklearn.model_selection import train_test_split
BLE_dataset,BLE_testset = train_test_split(df_BLE, test_size = 0.2, train_size = 0.8, random_state=0)
Zigbee_dataset,Zigbee_testset = train_test_split(df_Zigbee, test_size = 0.2, train_size = 0.8, random_state=0)
WiFi_dataset,WiFi_testset = train_test_split(df_WiFi, test_size = 0.2, train_size = 0.8, random_state=0)

### BLE Technology

In [40]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(BLE_dataset,BLE_testset)

#### N estimator = 10

In [41]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)

lenght = list(range(0,12))
x_vec_0 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_0.append(x)    
average_0 = Average(x_vec_0)
print('Error = ', average_0)

Error =  [1.29545589]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [42]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for BLE = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 10    MSE =  1.29545588832831
N Estimator = 100    MSE =  1.27675079735464


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.3086242854623693
Best N Estimator for BLE =  100


### Zigbee Technology

In [43]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(Zigbee_dataset,Zigbee_testset)

#### N estimator = 10

In [44]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.47354621]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [45]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for Zigbee = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 10    MSE =  1.473546206831793
N Estimator = 100    MSE =  1.458823134426608


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.423204481840889
Best N Estimator for Zigbee =  1000


### WiFi Technology

In [46]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(WiFi_dataset,WiFi_testset)

#### N estimator = 10

In [47]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.21781658]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [48]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for WiFi = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Use

N Estimator = 10    MSE =  1.2178165800442013
N Estimator = 100    MSE =  1.395232180630355


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.4284434539204847
Best N Estimator for WiFi =  10


## Scenario 2

In [49]:
#read dataset for scenario 2
df_BLE = pd.read_csv('BLE_scenario2.csv')
df_Zigbee = pd.read_csv('Zigbee_scenario2.csv')
df_WiFi = pd.read_csv('WiFi_scenario2.csv')

In [50]:
# TO SPLIT THE WHOLE DATASET INTO TRAINING DATASET AND TESTING DATASET
from sklearn.model_selection import train_test_split
BLE_dataset,BLE_testset = train_test_split(df_BLE, test_size = 0.2, train_size = 0.8, random_state=0)
Zigbee_dataset,Zigbee_testset = train_test_split(df_Zigbee, test_size = 0.2, train_size = 0.8, random_state=0)
WiFi_dataset,WiFi_testset = train_test_split(df_WiFi, test_size = 0.2, train_size = 0.8, random_state=0)

### BLE Technology

In [51]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(BLE_dataset,BLE_testset)

#### N estimator = 10

In [52]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.31677379]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [53]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for BLE = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 10    MSE =  1.316773791594861
N Estimator = 100    MSE =  1.4193948054968453


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.4292941879432235
Best N Estimator for BLE =  10


### Zigbee Technology

In [54]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(Zigbee_dataset,Zigbee_testset)

#### N estimator = 10

In [55]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.79746561]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [56]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for Zigbee = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Use

N Estimator = 10    MSE =  1.7974656141333796
N Estimator = 100    MSE =  1.7742957077712032


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.7505934028896704
Best N Estimator for Zigbee =  1000


### WiFi Technology

In [57]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(WiFi_dataset,WiFi_testset)

#### N estimator = 10

In [58]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.52903403]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [59]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for WiFi = ", n_est[av.index(min(av))])

N Estimator = 10    MSE =  1.5290340254838024


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 100    MSE =  1.4086469437516613


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.4204459824883005
Best N Estimator for WiFi =  100


## Scenario 3

In [60]:
#read dataset for scenario 3
df_BLE = pd.read_csv('BLE_scenario3.csv')
df_Zigbee = pd.read_csv('Zigbee_scenario3.csv')
df_WiFi = pd.read_csv('WiFi_scenario3.csv')

In [61]:
# TO SPLIT THE WHOLE DATASET INTO TRAINING DATASET AND TESTING DATASET
from sklearn.model_selection import train_test_split
BLE_dataset,BLE_testset = train_test_split(df_BLE, test_size = 0.2, train_size = 0.8, random_state=0)
Zigbee_dataset,Zigbee_testset = train_test_split(df_Zigbee, test_size = 0.2, train_size = 0.8, random_state=0)
WiFi_dataset,WiFi_testset = train_test_split(df_WiFi, test_size = 0.2, train_size = 0.8, random_state=0)

### BLE Technology

In [62]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(BLE_dataset,BLE_testset)

#### N estimator = 10

In [63]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.30134769]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [64]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   RMSE = ", ave)
    av.append(ave)
print("Best N Estimator for BLE = ", n_est[av.index(min(av))])

N Estimator = 10    RMSE =  1.3013476916305573


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Use

N Estimator = 100    RMSE =  1.6362650882373966


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    RMSE =  1.734488485462718
Best N Estimator for BLE =  10


### Zigbee Technology

In [65]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(Zigbee_dataset,Zigbee_testset)

#### N estimator = 10

In [66]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [2.25817331]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [67]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for Zigbee = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 10    MSE =  2.2581733113786147
N Estimator = 100    MSE =  1.8173776643692412


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


N Estimator = 1000    MSE =  1.8938262766314249
Best N Estimator for Zigbee =  100


### WiFi Technology

In [68]:
X_train, X_test, Y1_train, Y2_train, Y1_test, Y2_test = separateData(WiFi_dataset,WiFi_testset)

#### N estimator = 10

In [69]:
y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,10)
lenght = list(range(0,len(X_test)))
x_vec_1 = []
for i in lenght:
    x = error(y1_pred,Y1_test,y2_pred,Y2_test)
    x_vec_1.append(x)
average_1 = Average(x_vec_1)
print('Error = ', average_1)

Error =  [1.30462908]


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


###### Find best n estimator

In [ ]:
av = []
# create vector for N Estimators
n_est = [10, 100, 1000]
for i in n_est:
    y1_pred, y2_pred = RandomForest(X_train, X_test, Y1_train, Y2_train,i)
    ave = geterror(y1_pred,Y1_test,y2_pred,Y2_test)
    print('N Estimator =', i, "   MSE = ", ave)
    av.append(ave)
print("Best N Estimator for WiFi = ", n_est[av.index(min(av))])

C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.
C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Use

N Estimator = 10    MSE =  1.3046290824721019
N Estimator = 100    MSE =  1.501261163278443


C:\Users\kenshi410\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


# Summary

<img src="RF.jpg"/>

According to the results, the parameter with the lowest error for all technologies for scenario 1 is n estimator = 10,  
n estimator = 1000 for Scenario 2 and lastly, n estimator = 10 for scenario 3.

To select the best parameter for all scenario, we select n estimator with the lowest average deviation error. From the result, the parameter for all scenario is n estimator =  10 with an average error of 1.499360243 meters.

Overall, the Random Forest method deviated off from the original receiver position of 1.522363688 meters.